# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.chat_models import ChatOllama

# llm = OpenAI(temperature=0)
llm = ChatOllama(model="qwen:32b-chat-v1.5-q5_K_M")

In [7]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    print(relevant_parts)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")
print(doc_path)

['/']
['/', 'home']
['/', 'home', 'huangwei1']
['/', 'home', 'huangwei1', 'repo']
['/', 'home', 'huangwei1', 'repo', 'langchain']
['/', 'home', 'huangwei1', 'repo', 'langchain', 'cookbook']
/home/huangwei1/repo/langchain/cookbook/state_of_the_union.txt


In [9]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings


loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model="znbang/bge:large-zh-v1.5-f16")
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [10]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

In [12]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [13]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2134, which is longer than the specified 1000
Created a chunk of size 2470, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1636, which is longer than the specified 1000


## Create the Agent

In [14]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [15]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [16]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/home/huangwei1/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [17]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)

/home/huangwei1/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I should use the State of Union QA System to answer this.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the State of the Union address
Observation: In the State of the Union address, President Biden mentioned that he had nominated Judge Ketanji Brown Jackson four days prior to the speech. He described her as one of the nation's top legal minds and highlighted that she will continue Justice Stephen Breyer's legacy of excellence on the United States Supreme Court.
Thought:I now know the final answer.
Final Answer: President Biden mentioned Judge Ketanji Brown Jackson in his State of the Union address, noting that he had recently nominated her for the Supreme Court. He praised her as an outstanding legal mind and highlighted her potential to continue Justice Stephen Breyer's legacy on the bench.

> Finished chain.


"President Biden mentioned Judge Ketanji Brown Jackson in his State of the Union address, noting that he had recently nominated her for the Supreme Court. He praised her as an outstanding legal mind and highlighted her potential to continue Justice Stephen Breyer's legacy on the bench."

In [18]:
agent.run("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
I should use the Ruff QA System to answer this question since it's about a specific linter tool.
Action: Ruff QA System
Action Input: Why use ruff over flake8
Observation: Ruff can be used as a drop-in replacement for Flake8 when used without or with a small number of plugins, alongside Black, and on Python 3 code. Under those conditions, Ruff implements every rule in Flake8, including rules from Pyflakes (F), pycodestyle (E and W), and some popular Flake8 plugins like isort, yesqa, eradicate, and certain pyupgrade rules. However, it does not implement all the "opinionated" lint rules from flake8-bugbear or support custom lint rules like Flake8 does. Ruff's primary advantage lies in its performance and being a Rust-based alternative that can minimize conflicts across plugins by using different rule codes and allowing easy toggling of plugin rules. Additionally, it offers compatibility with Black for formatting, though the line-length enforcement 

"Ruff offers performance improvements over Flake8 and is a Rust-based alternative that can minimize conflicts between plugins by using different rule codes. It's compatible with Black for formatting, though there might be minor differences. For straightforward use cases without many custom lint rules or support for 'opinionated' lint rules, Ruff can serve as a viable drop-in replacement for Flake8 on Python 3 code. However, if you require extensive customization or rely on specific Flake8 plugins, you may need to weigh the benefits and limitations carefully."

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [19]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [20]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [21]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
I should use the State of Union QA System to find out what Biden said about Ketanji Brown Jackson in his speech.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the State of the Union address
Observation: In the State of the Union address, President Biden honored Judge Ketanji Brown Jackson, whom he nominated to serve on the United States Supreme Court. He described her as one of our nation's top legal minds and mentioned that she will continue Justice Stephen Breyer's legacy of excellence.


> Finished chain.


"In the State of the Union address, President Biden honored Judge Ketanji Brown Jackson, whom he nominated to serve on the United States Supreme Court. He described her as one of our nation's top legal minds and mentioned that she will continue Justice Stephen Breyer's legacy of excellence."

In [22]:
agent.run("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
I should use the Ruff QA system to get an answer specific to ruff and its comparison with flake8.
Action: Ruff QA System
Action Input: Why would someone choose ruff over flake8
Observation: Someone might choose Ruff over Flake8 if they are looking for a linter that is compatible with Black and designed to be used alongside a formatter. Ruff provides consistent line-length enforcement while working well with Black's formatting style, making it a suitable alternative for projects already using Black. Additionally, Ruff offers improved performance compared to Flake8 due to its implementation in Rust. However, Ruff doesn't support custom lint rules, so if that is a requirement, then Flake8 might be a better choice. Furthermore, Ruff's compatibility with Flake8 plugins is limited to a subset of the规则 and it does not support all "opinionated" lint rules from flake8-bugbear.


> Finished chain.


'Someone might choose Ruff over Flake8 if they are looking for a linter that is compatible with Black and designed to be used alongside a formatter. Ruff provides consistent line-length enforcement while working well with Black\'s formatting style, making it a suitable alternative for projects already using Black. Additionally, Ruff offers improved performance compared to Flake8 due to its implementation in Rust. However, Ruff doesn\'t support custom lint rules, so if that is a requirement, then Flake8 might be a better choice. Furthermore, Ruff\'s compatibility with Flake8 plugins is limited to a subset of the规则 and it does not support all "opinionated" lint rules from flake8-bugbear.'

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [23]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [24]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [25]:
agent.run(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
I need to find out which tool Ruff uses for Jupyter Notebooks and then check if the President mentioned it in the State of the Union address.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks
Observation: Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks. To use Ruff with Jupyter Notebooks, you can install both ruff and nbqa, then run commands like `nbqa ruff Untitled.ipynb` to lint and format the notebook.
Thought:Now I need to check if the President mentioned nbQA in the State of Union address.
Action: State of Union QA System
Action Input: Did the president mention nbQA in the state of the union
Observation: No, the president did not mention "nbQA" in the state of the union address.
Thought:I have found that Ruff uses nbQA for Jupyter Notebooks and confirmed that it was not mentioned by the President in the State of the Union.
Final Answer: Ruff uses nbQA t

'Ruff uses nbQA to run over Jupyter Notebooks, but the president did not mention nbQA during the most recent State of the Union address.'